<a href="https://colab.research.google.com/github/SegaObin/PET-projects/blob/main/titanic_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df.drop(['Cabin', 'Name', 'Ticket'], axis=1, inplace=True)

In [5]:
df.set_index('PassengerId', inplace=True)

In [6]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
df.isna().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2


In [8]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

In [9]:
df.Survived.value_counts()

,count
Survived,
0,549
1,342


# LogReg

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline


In [11]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


In [12]:
X = df.drop('Survived', axis=1)
y = df['Survived']

In [13]:
cat_features = ['Sex', 'Embarked']
num_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [14]:
scaler = StandardScaler()
ohe = OneHotEncoder()

In [15]:
X_scaled_df = pd.DataFrame(scaler.fit_transform(X[num_features]), index=X.index, columns=num_features)
X_ohe_df = pd.DataFrame(ohe.fit_transform(X[cat_features]).toarray(), index=X.index, columns=ohe.get_feature_names_out(cat_features))

X = pd.concat([X_scaled_df, X_ohe_df], axis=1)
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,
1,0.827377,-0.565736,0.432793,-0.473674,-0.502445,0.0,1.0,0.0,0.0,1.0
2,-1.566107,0.663861,0.432793,-0.473674,0.786845,1.0,0.0,1.0,0.0,0.0
3,0.827377,-0.258337,-0.474545,-0.473674,-0.488854,1.0,0.0,0.0,0.0,1.0
4,-1.566107,0.433312,0.432793,-0.473674,0.420730,1.0,0.0,0.0,0.0,1.0
5,0.827377,0.433312,-0.474545,-0.473674,-0.486337,0.0,1.0,0.0,0.0,1.0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43, shuffle=True)

In [17]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [18]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.76      0.85      0.80       110
           1       0.70      0.57      0.62        69

    accuracy                           0.74       179
   macro avg       0.73      0.71      0.71       179
weighted avg       0.73      0.74      0.73       179



# RandomForest

In [19]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


In [20]:
X = df.drop('Survived', axis=1)
y = df['Survived']

In [21]:
ohe = OneHotEncoder()
cat_features

['Sex', 'Embarked']

In [22]:
X_ohe_df = pd.DataFrame(ohe.fit_transform(X[cat_features]).toarray(), index=X.index, columns=ohe.get_feature_names_out(cat_features))

In [23]:
X.drop(cat_features, axis=1, inplace=True)

In [24]:
X = pd.concat([X, X_ohe_df], axis=1)

In [25]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,
1,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
2,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
3,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
4,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
5,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [27]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [28]:
preds = model.predict(X_test)
report = classification_report(y_test, y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.59      0.69      0.63       105
           1       0.41      0.31      0.35        74

    accuracy                           0.53       179
   macro avg       0.50      0.50      0.49       179
weighted avg       0.51      0.53      0.52       179



# NN 1

In [29]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


In [30]:
train, test = train_test_split(df, test_size=0.2, shuffle=True)

In [31]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
446,1,1,male,4.0,0,2,81.8583,S
84,0,1,male,28.0,0,0,47.1000,S
411,0,3,male,28.0,0,0,7.8958,S
870,1,3,male,4.0,1,1,11.1333,S
57,1,2,female,21.0,0,0,10.5000,S


In [32]:
cat_features = ['Sex', 'Embarked']
num_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [33]:
scaler = StandardScaler()
ohe = OneHotEncoder()
train_target = train['Survived']
train_scaled = pd.DataFrame(scaler.fit_transform(train[num_features]), index=train.index, columns=num_features)
train_ohe = pd.DataFrame(ohe.fit_transform(train[cat_features]).toarray(), index=train.index, columns=ohe.get_feature_names_out(cat_features))

train = pd.concat([train_scaled, train_ohe, train_target], axis=1)
train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived
PassengerId,,,,,,,,,,,
446,-1.529798,-1.950676,-0.476988,2.026010,0.905660,0.0,1.0,0.0,0.0,1.0,1
84,-1.529798,-0.091622,-0.476988,-0.478363,0.249943,0.0,1.0,0.0,0.0,1.0,0
411,0.856218,-0.091622,-0.476988,-0.478363,-0.489646,0.0,1.0,0.0,0.0,1.0,0
870,0.856218,-1.950676,0.423849,0.773823,-0.428571,0.0,1.0,0.0,0.0,1.0,1
57,-0.336790,-0.633846,-0.476988,-0.478363,-0.440518,1.0,0.0,0.0,0.0,1.0,1


In [34]:
test_target = test['Survived']
test_scaled = pd.DataFrame(scaler.transform(test[num_features]), index=test.index, columns=num_features)
test_ohe = pd.DataFrame(ohe.transform(test[cat_features]).toarray(), index=test.index, columns=ohe.get_feature_names_out(cat_features))

test = pd.concat([test_scaled, test_ohe, test_target], axis=1)
test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived
PassengerId,,,,,,,,,,,
865,-0.336790,-0.401464,-0.476988,-0.478363,-0.393355,0.0,1.0,0.0,0.0,1.0,0
487,-1.529798,0.450602,0.423849,-0.478363,1.059254,1.0,0.0,0.0,0.0,1.0,1
161,0.856218,1.147747,-0.476988,0.773823,-0.334873,0.0,1.0,0.0,0.0,1.0,0
844,0.856218,0.411872,-0.476988,-0.478363,-0.517157,0.0,1.0,1.0,0.0,0.0,0
770,0.856218,0.218220,-0.476988,-0.478363,-0.480842,0.0,1.0,0.0,0.0,1.0,0


In [35]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived']

X_test = test.drop('Survived', axis=1)
y_test = test['Survived']

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [37]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# NN 1

In [38]:
class Net1(nn.Module):
  def __init__(self, input_size):
    super(Net1, self).__init__()
    self.model = nn.Sequential(
        nn.Linear(input_size, 32),
        nn.ReLU(),

        nn.Linear(32, 16),
        nn.ReLU(),

        nn.Linear(16, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.model(x)

input_dim = X_train_tensor.shape[1]
model = Net1(input_dim)

In [39]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 100

for epoch in range(epochs):
  model.train()
  running_loss = 0.0

  for batch_X, batch_y in train_loader:
    optimizer.zero_grad()
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  if (epoch + 1) % 10 == 0:
    print(f'Epoch {epoch}, loss {running_loss/len(train_loader)}')

Epoch 9, loss 0.6623489856719971
Epoch 19, loss 0.6293486745461173
Epoch 29, loss 0.5779549272164054
Epoch 39, loss 0.5103538334369659
Epoch 49, loss 0.462191641330719
Epoch 59, loss 0.42972131267837854
Epoch 69, loss 0.42686953233635944
Epoch 79, loss 0.42053860231586127
Epoch 89, loss 0.41292125116223877
Epoch 99, loss 0.40968733896379883


In [40]:
model.eval()
all_preds = []

with torch.no_grad():
  test_outputs = model(X_test_tensor)
  test_preds = (test_outputs > 0.5)

In [41]:
print(classification_report(y_test_tensor, test_preds))

              precision    recall  f1-score   support

         0.0       0.81      0.89      0.85       115
         1.0       0.75      0.62      0.68        64

    accuracy                           0.79       179
   macro avg       0.78      0.76      0.77       179
weighted avg       0.79      0.79      0.79       179



# NN 2

In [42]:
class Net2(nn.Module):
  def __init__(self, input_size):
    super(Net2, self).__init__()
    self.model = nn.Sequential(
        nn.Linear(input_size, 32),
        nn.Tanh(),
        nn.Dropout(0.2),

        nn.Linear(32, 16),
        nn.Tanh(),
        nn.Dropout(0.2),

        nn.Linear(16, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.model(x)

In [43]:
input_dim = X_train_tensor.shape[1]
criterion = nn.BCELoss()

mdoel = Net2(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [44]:
epochs = 100

for epoch in range(epochs):
  model.train()
  running_loss = 0.0

  for batch_X, batch_y in train_loader:
    optimizer.zero_grad()
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)

    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  if (epoch + 1) % 10 == 0:
    print(f'Epoch {epoch}, loss {running_loss/len(train_loader)}')

Epoch 9, loss 0.3910929631927739
Epoch 19, loss 0.38126996483491815
Epoch 29, loss 0.3658294820267221
Epoch 39, loss 0.3556095743956773
Epoch 49, loss 0.35597701565079065
Epoch 59, loss 0.3432547845270323
Epoch 69, loss 0.3416413820308188
Epoch 79, loss 0.3349137526491414
Epoch 89, loss 0.3282666582128276
Epoch 99, loss 0.3235189364008281


In [45]:
model.eval()

with torch.no_grad():
  test_outputs = model(X_test_tensor)
  test_preds = (test_outputs > 0.5)

In [46]:
print(classification_report(y_test_tensor, test_preds))

              precision    recall  f1-score   support

         0.0       0.80      0.92      0.86       115
         1.0       0.81      0.59      0.68        64

    accuracy                           0.80       179
   macro avg       0.81      0.76      0.77       179
weighted avg       0.80      0.80      0.80       179



# Submission

In [47]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [48]:
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

In [49]:
test.set_index('PassengerId', inplace=True)
test.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_scaled = pd.DataFrame(scaler.transform(test[num_features]), index=test.index, columns=num_features)
test_ohe = pd.DataFrame(ohe.transform(test[cat_features]).toarray(), index=test.index, columns=ohe.get_feature_names_out(cat_features))

test = pd.concat([test_scaled, test_ohe], axis=1)
test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,
892,0.856218,0.411872,-0.476988,-0.478363,-0.490903,0.0,1.0,0.0,1.0,0.0
893,0.856218,1.380129,0.423849,-0.478363,-0.506545,1.0,0.0,0.0,0.0,1.0
894,-0.336790,2.542037,-0.476988,-0.478363,-0.455846,0.0,1.0,0.0,1.0,0.0
895,0.856218,-0.169083,-0.476988,-0.478363,-0.475182,0.0,1.0,0.0,0.0,1.0
896,0.856218,-0.556385,0.423849,0.773823,-0.406796,1.0,0.0,0.0,0.0,1.0


In [50]:
test_tensor = torch.tensor(test.values, dtype=torch.float32)
sub_probs = model(test_tensor)
sub_preds = (sub_probs > 0.5).to(torch.int32)

In [51]:
sub_preds.view(-1)

tensor([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,

In [52]:
submit = pd.DataFrame(
    {
     'PassengerId' : test.index,
     'Survived' : sub_preds.view(-1)
    })
submit.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [53]:
submit.to_csv('submit.csv', index=False)